In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/diabetes-dataset/diabetes.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix, 
    classification_report, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score
)

In [3]:
df = pd.read_csv("/kaggle/input/diabetes-dataset/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [6]:
print(df.isna().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [7]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']
    
# Excluding column Pregnancies where 0 is a valid value
zero_columns = X.columns[~X.columns.isin(['Pregnancies'])]  
X[zero_columns] = X[zero_columns].replace(0, np.nan)
    
# Fill NaN values with mean
X = X.fillna(X.mean())

In [8]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

In [10]:
#no hyperparams : defaults
model_default = LogisticRegression(max_iter=1000)
model_default.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [11]:
y_pred = model_default.predict(X_test)
    
# Metrics
def evaluate_model(y_pred,y_test):
    print("-"*40)
    print("Metrics:\n")
    print("\t Accuracy : {:.3f}".format(accuracy_score(y_test, y_pred)))
    print("\t Precision : {:.3f}".format(precision_score(y_test, y_pred)))
    print("\t Recal : {:.3f}".format(recall_score(y_test, y_pred)))
    print("\t F1 Score : {:.3f}".format(f1_score(y_test, y_pred)))
    print("-"*40)

In [12]:
#default params scores
evaluate_model(y_pred,y_test)


----------------------------------------
Metrics:

	 Accuracy : 0.753
	 Precision : 0.667
	 Recal : 0.618
	 F1 Score : 0.642
----------------------------------------


In [13]:
# decent scores but lets see if we can tune it
from sklearn.model_selection import GridSearchCV

model = LogisticRegression(max_iter=1000)
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  
    'penalty': ['l1', 'l2'],       
    'solver': ['liblinear', 'saga'] 
}

model_GSCV = GridSearchCV(model,param_grid,cv=5,scoring="accuracy",n_jobs=-1)
model_GSCV.fit(X_train,y_train)

best_model = model_GSCV.best_estimator_

In [14]:
y_pred_GSCV = best_model.predict(X_test)

evaluate_model(y_pred_GSCV,y_test)

----------------------------------------
Metrics:

	 Accuracy : 0.779
	 Precision : 0.723
	 Recal : 0.618
	 F1 Score : 0.667
----------------------------------------


In [15]:
print("Best Parameters:", model_GSCV.best_params_)
print("Best Accuracy:", model_GSCV.best_score_)

Best Parameters: {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}
Best Accuracy: 0.7703718512594963


#  Our best score for Logistic Regression is  0.77 

In [16]:
#is it enough ? ... let's see with other models

# 1 - Random  Forest
from sklearn.ensemble import RandomForestClassifier


rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', None]
}


rf_model = RandomForestClassifier(random_state=42)


rf_grid_search = GridSearchCV(rf_model, rf_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
rf_grid_search.fit(X_train, y_train)


best_rf = rf_grid_search.best_estimator_
rf_y_pred = best_rf.predict(X_test)


evaluate_model(rf_y_pred, y_test)


print("Best RF Params:", rf_grid_search.best_params_)


----------------------------------------
Metrics:

	 Accuracy : 0.760
	 Precision : 0.655
	 Recal : 0.691
	 F1 Score : 0.673
----------------------------------------
Best RF Params: {'class_weight': None, 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50}


In [17]:
#svc ??
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline


svc_param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__kernel': ['linear', 'rbf', 'poly'],
    'svc__gamma': ['scale', 'auto']
}


svc_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(probability=True, random_state=42))
])


svc_grid_search = GridSearchCV(svc_pipeline, svc_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
svc_grid_search.fit(X_train, y_train)


best_svc = svc_grid_search.best_estimator_
svc_y_pred = best_svc.predict(X_test)


evaluate_model(svc_y_pred, y_test)


print("Best SVC Params:", svc_grid_search.best_params_)


----------------------------------------
Metrics:

	 Accuracy : 0.753
	 Precision : 0.681
	 Recal : 0.582
	 F1 Score : 0.627
----------------------------------------
Best SVC Params: {'svc__C': 1, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}


In [18]:
#voting Classifier with the three models combined ?
from sklearn.ensemble import VotingClassifier


logreg = LogisticRegression(C=10, penalty='l1', solver='liblinear', max_iter=1000, random_state=42)
rf = RandomForestClassifier(n_estimators=50, max_depth=10, min_samples_leaf=4, min_samples_split=10, random_state=42)
svc = SVC(C=1, kernel='rbf', gamma='scale', probability=True, random_state=42)


voting_clf = VotingClassifier(estimators=[
    ('Logistic Regression', logreg),
    ('Random Forest', rf),
    ('SVC', svc)
], voting='soft')  


voting_clf.fit(X_train, y_train)
y_pred_voting = voting_clf.predict(X_test)


evaluate_model(y_pred_voting, y_test)


----------------------------------------
Metrics:

	 Accuracy : 0.760
	 Precision : 0.673
	 Recal : 0.636
	 F1 Score : 0.654
----------------------------------------


In [19]:
#XGBoost ??
from xgboost import XGBClassifier

# Define XGBoost model
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# Train
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate
evaluate_model(y_pred_xgb, y_test)


----------------------------------------
Metrics:

	 Accuracy : 0.734
	 Precision : 0.617
	 Recal : 0.673
	 F1 Score : 0.643
----------------------------------------


In [20]:
#Gradient Boost ?
from sklearn.ensemble import GradientBoostingClassifier


gb_tuned = GradientBoostingClassifier(
    n_estimators=150,     
    learning_rate=0.05,   
    max_depth=4,          
    min_samples_split=10, 
    min_samples_leaf=4,   
    subsample=0.8,        
    max_features='sqrt',  
    random_state=42
)

param_grid_gb = {
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 4, 5],
    'n_estimators': [100, 150]
}

gb_grid = GridSearchCV(gb_tuned, param_grid_gb, cv=5, scoring='accuracy', n_jobs=-1)
gb_grid.fit(X_train, y_train)
best_gb = gb_grid.best_estimator_
y_pred_gb = best_gb.predict(X_test)

evaluate_model(y_pred_gb,y_test)


----------------------------------------
Metrics:

	 Accuracy : 0.740
	 Precision : 0.627
	 Recal : 0.673
	 F1 Score : 0.649
----------------------------------------


# Our Tuned Logistic regression had the best accuracy score ~ 0.80 and other models had variant best Recall/precision scores ...